#1 张量创建：从无到有构建基础单元

tesor是PyTorch的核心数据结构，类似NumPy数组，但支持GPU加速和自动微分。以下是最常用的创建方式：

| op  | 功能描述| 关键参数 |
|----------|----------|----------|
|torch.tensor|	从列表/元组/NumPy数组转换为张量|	data(输入数据)、requires_grad(是否追梯度)|
|torch.zeros|	生成全0张量	|size(张量形状，如(2,3))|
|torch.ones|	生成全1张量	|size(张量形状)|
|torch.rand|	生成[0,1)均匀分布的随机张量	|size(张量形状)|
|torch.randn|	生成均值0、标准差1的正态分布随机张量	|size(张量形状)|
|torch.arange|	生成等差数列张量（类似Python的range）|	start(起始值)、end(终止值)、step(步长)|
|torch.linspace|	生成固定数量的均匀间隔张量（包含终点）	|start、end、steps(间隔数)|
|torch.logspace|	生成对数尺度的均匀间隔张量	|start、end、steps、base(对数基，默认10)|
|torch.eye|	生成单位矩阵（对角线1，其余0）	|n(行数)、m(列数，默认等于n)|
|torch.empty|	生成未初始化张量（内存残留值，速度快）| size(张量形状)|
|torch.full|	生成全为指定值的张量	|size(形状)、fill_value(填充值）|



In [2]:
import torch

# 1. 从列表创建张量，指定追梯度
# 只有浮点数才可以设置梯度
a = torch.tensor([1.0, 2, 3], requires_grad=True)
print("torch.tensor创建：", a)  # 输出：tensor([1, 2, 3], requires_grad=True)

# 2. 生成2行3列的全0张量
b = torch.zeros(2, 3)
print("torch.zeros创建：\n", b)  # 输出2x3的0矩阵

# 3. 生成0到10、步长2的等差数列
c = torch.arange(0, 10, 2)
print("torch.arange创建：", c)  # 输出：tensor([0, 2, 4, 6, 8])

torch.tensor创建： tensor([1., 2., 3.], requires_grad=True)
torch.zeros创建：
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
torch.arange创建： tensor([0, 2, 4, 6, 8])


#2 元素级运算：逐元素的加减乘除与函数
对张量中每个元素单独操作，要求输入张量形状相同（或满足广播条件）。
##2.1 基础算术运算
# PyTorch 逐元素数学运算速查表

## 📌 基础操作对照表

| 操作        | 功能描述                     | 语法示例                                 | 补充说明 |
|-------------|------------------------------|------------------------------------------|--------|
| `torch.add` | 逐元素加法                   | `torch.add(a, b, alpha=2)`               | 等价于 `a + alpha * b`；`alpha` 可选，默认为1 |
| `torch.sub` | 逐元素减法                   | `torch.sub(a, b)`                        | 等价于 `a - b` |
| `torch.mul` | 逐元素乘法（非矩阵乘）       | `torch.mul(a, b)`                        | 等价于 `a * b`；注意不是 `torch.mm` 或 `@` |
| `torch.div` | 逐元素除法                   | `torch.div(a, b)`                        | 默认为浮点除法；可指定 `rounding_mode='floor'` 或 `'trunc'` |
| `torch.pow` | 逐元素幂运算                 | `torch.pow(a, 2)`                        | 支持标量或张量作为指数，如 `torch.pow(a, b)` |
| `torch.sqrt`| 逐元素平方根                 | `torch.sqrt(a)`                          | 输入必须 ≥ 0，否则返回 `nan` |
| `torch.exp` | 逐元素自然指数（$e^x$）     | `torch.exp(a)`                           | 常用于激活函数（如 Softmax）前处理 |
| `torch.log` | 逐元素自然对数（$\ln x$）   | `torch.log(a)`                           | 要求 `a > 0`，否则返回 `nan` 或 `-inf` |

---

## 💡 使用小贴士

- **广播支持**：所有上述操作都支持广播（broadcasting），即不同形状的张量在满足广播规则时可自动对齐。
- **In-place 操作**：大多数函数有 in-place 版本（以 `_` 结尾），例如 `a.add_(b)` 会直接修改 `a`，节省内存但需谨慎使用。
- **数值稳定性**：使用 `torch.log` 或 `torch.sqrt` 时，确保输入满足定义域要求，避免出现 `nan` 或 `inf`。

In [3]:
import torch

a = torch.tensor([1.0, 4.0, 9.0])
b = torch.tensor([2.0, 2.0, 2.0])

print(torch.add(a, b, alpha=0.5))   # 输出: tensor([ 2.,  5., 10.])
print(torch.mul(a, b))              # 输出: tensor([ 2.,  8., 18.])
print(torch.sqrt(a))                # 输出: tensor([1., 2., 3.])
print(torch.log(a))                 # 输出: tensor([0.0000, 1.3863, 2.1972])
a.add_(b)
print(a)                            # 输出: tensor([ 3.,  6., 11.])

tensor([ 2.,  5., 10.])
tensor([ 2.,  8., 18.])
tensor([1., 2., 3.])
tensor([0.0000, 1.3863, 2.1972])
tensor([ 3.,  6., 11.])


##2.2 统计类运算
### 📊 基础统计运算

| 操作 | 功能描述 | 语法示例 | 补充说明 |
|------|----------|----------|----------|
| `torch.sum` | 计算张量元素总和 | `torch.sum(a)` / `torch.sum(a, dim=0)` | 支持沿特定维度求和，`keepdim=True`保留维度 |
| `torch.mean` | 计算张量元素均值 | `torch.mean(a)` / `torch.mean(a, dim=1)` | 数据类型必须为浮点型，否则报错 |
| `torch.std` | 计算标准差 | `torch.std(a, unbiased=True)` | `unbiased=False`使用总体标准差 |
| `torch.var` | 计算方差 | `torch.var(a, unbiased=True)` | 与 `std` 类似，可选无偏估计 |
| `torch.min` | 最小值及索引 | `torch.min(a)` / `torch.min(a, dim=1)` | 返回 `(values, indices)` 元组 |
| `torch.max` | 最大值及索引 | `torch.max(a)` / `torch.max(a, dim=0)` | 返回 `(values, indices)` 元组 |
| `torch.argmin` | 最小值索引 | `torch.argmin(a, dim=0)` | 返回索引位置，不返回值本身 |
| `torch.argmax` | 最大值索引 | `torch.argmax(a, dim=1)` | 返回索引位置，不返回值本身 |

---

### 🧮 高级统计运算

| 操作 | 功能描述 | 语法示例 | 补充说明 |
|------|----------|----------|----------|
| `torch.median` | 中位数 | `torch.median(a)` / `torch.median(a, dim=1)` | 返回 `(values, indices)` 元组 |
| `torch.mode` | 众数 | `torch.mode(a, dim=0)` | 返回 `(values, indices)` 元组 |
| `torch.quantile` | 分位数 | `torch.quantile(a, q=0.5)` | `q` 为分位点 (0~1)，仅支持一维张量或指定维度 |
| `torch.topk` | 最大的 k 个值 | `torch.topk(a, k=3, dim=1)` | 返回 `(values, indices)` 元组，按降序排列 |
| `torch.sort` | 排序 | `torch.sort(a, dim=0, descending=False)` | 返回 `(values, indices)` 元组 |
| `torch.unique` | 去重并排序 | `torch.unique(a, return_counts=True)` | 可选择返回计数 (`return_counts=True`) |

---

### 💡 使用小贴士

- **维度参数**：大部分统计函数支持 `dim` 参数，用于沿特定轴计算。不指定时默认对全部元素操作。
- **保持维度**：使用 `keepdim=True` 可保留输出张量的维度信息，便于后续计算。
- **数据类型**：`mean`、`std`、`var` 等函数要求输入为浮点类型，整数张量需先转换（如 `.float()`）。
- **NaN 处理**：统计函数通常会传播 NaN 值（如含 NaN 的区域结果为 NaN），可用 `torch.nanmean` 等函数忽略 NaN。

---
> ⚠️ 注意：某些函数（如 `quantile`）在旧版 PyTorch 中可能不可用，建议检查版本兼容性。